# Array Programming Paradigm (Einstein's Notation)

## Overview

* We have seen Iverson's Notation
* And its implementation in Numpy
* Take Away:
  - Every value is an array
  - Operator as function of functions
  - Map-Reduce pattern: Matrix algorithms / Graph algorithms
* Generalize to N-Dimensions: Tensor functions
  - Growing bags of API and functions
    - Lots of mental burdens
    - Not portable among different implementation frameworks
    - Error prone
  - Is there a notation that rules them all?
  - Of course we have Einstein who likes to have theory of everything
  - Einstein Notation: Not invented, by popularized by Einstein

## Basic Einsum Functions

In [3]:
import numpy as np

A = np.arange(9).reshape( 3, 3 )
B = np.arange(9).reshape( 3, 3 )
print( A )
print( B )

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[0 1 2]
 [3 4 5]
 [6 7 8]]


Let's Look at Matric Multiplication again. Below is a typical imperative code. 

In [7]:
N = A.shape[0]
C = np.zeros( (N,N), dtype=int )
for i in range(N) :
    for j in range(N) :
        for k in range( N ) :
            C[i,j] += A[i,k] * B[k,j]

This works, but Einstein stared at it and figure out that the loop nest is really not necessary and can be made *implicit*. Also if we know we keep doing map-reduce on a specif semi-ring (don't worry about the math jagon, but just say it is map-reduce on *, +), we don't need them explicit either. This just leaves us the subscript! 

So we cut the verbosity and it becomes as terse as the following: 

ik, kj -> ij     

Someone (We actually should give credit to Mark Wiebe, the author) actually implemented the notation in Numpy and call it the einsum (Einstein Sum) function. Einsum looks like the following:

~~~
result = np.einsum( format, arg1, arg2, ..., argn )
~~~

format is a string using the Einsein notation, and gives the complete spec of a function over arg1, arg2, ... argn, each a N-D (tensor) value; and returns another tensor value. 

Of course the format string has to match the dimensions:

~~~
s = np.einsum( "a->", v )
T = np.einsum( "ij->ji", M )
C = np.einsum( "mn,np->mp", A, B )

assert v.ndim == len( "a" )
assert s.ndim == len( "" )

assert M.ndim == len( "ij" )
assert T.ndim == len( "ji" )

assert A.ndim == len( "mn" )
assert B.ndim == len( "np" )
assert C.ndim == len( "mp" )
~~~

Comparing to its nested loop counterpart, the einsum function is super compact!

Let's first see a few examples to get a flavour.

### 2-D Matrix Diagonal Extraction

In [8]:
d = np.zeros( (N), dtype=int )
for i in range( N ) :
    d[i] += A[i,i]

d2 = np.einsum( "ii->i", A )
print( A )
print( d )
print( d2 )

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[0 4 8]
[0 4 8]


### 2-D Matrix Trace

Recall the term "trace" in Linear Algebra? 

In [9]:
d = 0
for i in range( N ) :
    d += A[i,i]
d2 = np.einsum( "ii->", A )

print( A )
print( d )
print( d2 )
assert np.allclose( d, d2 )

[[0 1 2]
 [3 4 5]
 [6 7 8]]
12
12


### Three Argument Example

In [10]:
v = np.arange( N )
d = 0
for s in range( N ) :
    for t in range( N ) :
        d += v[s] * A[s,t] * v[t]
d2 = np.einsum( "s,st,t->", v, A, v )

print( v )
print( d )
print( d2 )
assert np.allclose( d, d2 )

[0 1 2]
60
60


### Batched Outer Product

In [11]:
d = np.zeros( (N,N,N), dtype=int )
for b in range( N ) :
    for i in range( N ) :
        for j in range( N ) :
            d[b,i,j] += A[b,i] * B[b,j]
d2 = np.einsum( "bi,bj->bij", A, B )

assert np.allclose( d, d2 ) 

### Semantics

In the above examples, we didn't care how functions are named and called, we just use a *canonical notation* and gets the work done. So how exactly an einsum is evaluated, in other words, what is its semantics (meaning)? 

We distingush two type of indices:

1. Free Indices:

   Indices *present* in the output indices;
   
1. Summation (Reduction) Indices: 

    Indices *disappeared* in the output indices.

Typically, free indices are in the outer loop, summation indices are in the inner loop.
As you recall, summation (reduction) carries out *dimension reduction*: that dimension will be GONE at the output. 

Now let's go back to the previous examples again and identify the free and summation indices. 

Also note that Einsum does NOT impose any evaluation order. In fact, the order of the indices in the format string does not matter at all, only their precense, and whether they the index label is repeated matters (that means axis of different inputs are aligned)! 

That said, the different evaluation order matters a lot in the implementation (recall the parallel map-reduce example?), but that's somebody else job, and at this stage, we make sure we have the right abstraction (so we don't OVER specify), and give the implementation frameworks enough freedom to do their job well.

There is a special case when we leave out the arrow ('->'). In this case, Numpy einsum will take the labels that appeared once and arrange them in alphabetical order. So 'ij, jk' is equivalent to 'ij, ji -> ik'.

In [8]:
print( np.einsum( 'ji', A ) )
print( np.einsum( 'ji->ij', A ) )

[[0 3 6]
 [1 4 7]
 [2 5 8]]
[[0 3 6]
 [1 4 7]
 [2 5 8]]


### Wrong Uses

~~~
#    Bad. Number of input index groups doesn't match number of
#         arguments.
np.einsum("ab,bc->ac", A)
 
#    Bad. Indexes must be ASCII upper/lowercase letters.
np.einsum("012,1^%->:;?", A, B)
 
#    Bad. Argument 0 has 3 dimensions but only 2 indexes are
#         given.
A = np.random.normal(size = (2,3,4))
B = np.random.normal(size = (4,5,6))
np.einsum("ab,bcd->a", A, B)
 
#    Bad. One of the output indexes isn't in the set of all
#         input indexes.
np.einsum("ab,bc->acz", A, B)
 
#    Bad. Output has a repeated index.
np.einsum("ab,bc->baa", A, B)


#    Bad. Mismatches in the sizes of input argument axes
#         that are labelled with the same index.
A = np.random.normal(size = (2,3,4))
B = np.random.normal(size = (3,4,5))
np.einsum("ckj,cqq->c", A, B)
~~~

### Canonical Notation

By Canonical we mean there is just ONE way of express something. 

Let's first look at 1D Vector functions.

| Einsum             | Numpy Equivalent  | Description         |
| ------------------ | ----------------- | ---                 |
| ('i', A)           | A                 | return a view of A  |
| ('i->', A)         | sum(A)            | sum all elements of A |
| ('i,i->i, A, B)    | A*B               | element-wise mult of two vectors |
| ('i,i->', A, B)    | inner(A, B)       | inner product of two vectors |
| ('i,j->ij', A, B)  | outer(A,B)        | outer product of two vectors |

Now let's look at 2D Matrix functions over matrices with compatible shapes.

| Einsum             | Numpy Equivalent  | Description         |
| ------------------ | ----------------- | ---                 |
| ('ij', A)          | A                 | a View of itself    |
| ('ji', A)          | A.T               | Transpose | 
| ('ii->i', A)       | diag(A)           | main diagnoal of a matrix |
| ('ij->', A)        | sum(A)            | reduction on ALL elements |
| ('ij->j', A)       | sum(A,axis=0)     | reduction on ALL rows |
| ('ij->i', A)       | sum(A,axis=1)     | reduction on ALL columns | 
| ('ij,ij->ij', A, B) |	A * B	         | Hadamard product: element-wise multiplication of A and B |
| ('ij,ji->ij', A, B) |	A * B.T	         | element-wise multiplication of A and B.T |
| ('ij,jk', A, B)	  | dot(A, B)	     | matrix multiplication of A and B |
| ('ij,kj->ik', A, B) | inner(A, B)	     | inner product of A and B |
| ('ij,kj->ikj', A, B) | A[:, None] * B	  | each row of A multiplied by B |
| ('ij,kl->ijkl', A, B)	| A[:, :, None, None] * B	| each value of A multiplied by B|




### Marching to Higher Dimensions

Einsum really shines when dealing with tensors beyond 2 dimensions.

* Batch Matrix Multiplication

In [12]:
# Batch matrix multiplication
X = np.arange(24).reshape(2, 3, 4)
Y = np.arange(40).reshape(2, 4, 5)

A = np.einsum('ijk, ikl->ijl', X, Y)

print(A)

[[[  70   76   82   88   94]
  [ 190  212  234  256  278]
  [ 310  348  386  424  462]]

 [[1510 1564 1618 1672 1726]
  [1950 2020 2090 2160 2230]
  [2390 2476 2562 2648 2734]]]


In this example, i,j,l are free indices, and k is the summation index. We extend our familiar matrix multiplication like behavior to three dimensional arrays. It can be thought of performing matrix multiplications on batches of data, hence the name. But in Einstin notation, there is really not much difference.

* Tensor Contraction

In [4]:
X = np.random.rand( 2,3,5,7 )
Y = np.random.rand( 11,13,3,17,5 )
A = np.einsum('pqrs,tuqvr->pstuv', X, Y)
print( A.shape )

(2, 7, 11, 13, 17)


## Battery: Einops, or Generalized Einsum

While Numpy einsum function is pioneering and extremely useful, it has a few limitations: 

1. It does not support reduction on arbitary functions. Only on addition.
2. Only allows on character to name axis, and gets error-prone for complex problems;
3. Implicit indexing relies on character sorting, and contradict the spirit of making explicit declarations.

Inspired on Numpy Einsum function, Alex Rogozhnikov implemented the Einops package, which revised a few design decisions regarding the Einsum Function itself. 

1. Use full identifier as name of axis, and use space as deliminator;
2. Format string is at the end (not at the beginning);
3. Implicit indexing is forbidden.

You could download the package yourself. 

~~~
%git clone https://github.com/arogozhnikov/einops.git
~~~

In [6]:
from einops import einsum

einsum( A, 'row col -> row' )

array([ 3, 12, 21])

In [7]:
einsum( A, B, 'i k, k j -> i j' )

array([[ 15,  18,  21],
       [ 42,  54,  66],
       [ 69,  90, 111]])

It also added explicit reduction function on arbitary functions and also a careful thoughtout, but very small set of tensor "rearranging" construct.

Next We will now go directly to the tutorial from the package documentation itself.

## Take Away

Einstein's notation, in the same spirit of Iverson's APL notation, encourages abstract thinking, and reduces a plethora of seemingly different things into a handful of concepts.

With engineering efforts progressed through Numpy and Einops (2018), the abstract thinking can translate to real world impact. For example, Einops not only admits extremely terse expression of your design intent, it can also connect transparently to different "backends". This not only incldues numpy, but also different deep learning frameworks, which map the program to different hardware, some can be as large as an entire cluster in a data center with tens of thought GPUs. 